Con este notebook de python lo que se ha hecho es a partir de la bbdd de access en formato mbd, se exportan todas las tablas a csv, tanto las principales como las auxiliares ya que se pierden las relaciones.
Con este script se rehace la bbdd haciendo joins en primer lugar a las dos tablas principales que son ListaAhogamientos y luego PersonasAhogadas.

En segundo lugar se hace un segundo join entre ambas tablas ya que tiene un foreign key hacia IdAhogado ya que en cada ahogamiento puede haber 1 o mas personas implicadas

In [41]:
import pandas as pd
import os
import sqlite3

Recorro todas las tablas de la carpeta y hago un bbdd de sqlite con todas las tablas

In [42]:
folder_path = '../tablas/csv'

txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
dataframes = {}

for txt_file in txt_files:
    file_name = os.path.splitext(txt_file)[0]  # sin extensión
    file_path = os.path.join(folder_path, txt_file)
    df = pd.read_csv(file_path, sep=';', encoding='latin1')
    dataframes[file_name] = df

conn = sqlite3.connect("ahogamientos.db") 

for table_name, df in dataframes.items():
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Tabla '{table_name}' insertada en SQLite")

conn.close()


Tabla 'Aux-Actividad' insertada en SQLite
Tabla 'Aux-Antecedentes' insertada en SQLite
Tabla 'Aux-Causas' insertada en SQLite
Tabla 'Aux-CCAA' insertada en SQLite
Tabla 'Aux-Deteccion' insertada en SQLite
Tabla 'Aux-Extraccion' insertada en SQLite
Tabla 'Aux-FactorRiesgo' insertada en SQLite
Tabla 'Aux-Intervencion' insertada en SQLite
Tabla 'Aux-Localizacion' insertada en SQLite
Tabla 'Aux-MaterialRescate' insertada en SQLite
Tabla 'Aux-Origen' insertada en SQLite
Tabla 'Aux-PrimerInterviniente' insertada en SQLite
Tabla 'Aux-Pronostico' insertada en SQLite
Tabla 'Aux-Provincia' insertada en SQLite
Tabla 'Aux-Reanimacion' insertada en SQLite
Tabla 'Aux-Riesgo' insertada en SQLite
Tabla 'Aux-Tipo' insertada en SQLite
Tabla 'AuxVigilancia' insertada en SQLite
Tabla 'ListaAhogamientos' insertada en SQLite
Tabla 'PersonasAhogadas' insertada en SQLite


Aqui vemos los nombres de las tablas principales

In [43]:
conn = sqlite3.connect("ahogamientos.db")

cursor = conn.cursor()
cursor.execute("PRAGMA table_info('ListaAhogamientos');")
columns_lista_ahogamientos = [column[1] for column in cursor.fetchall()]
print("Columnas de ListaAhogamientos:", columns_lista_ahogamientos)

cursor.execute("PRAGMA table_info('PersonasAhogadas');")
columns_personas_ahogadas = [column[1] for column in cursor.fetchall()]
print("Columnas de PersonasAhogadas:", columns_personas_ahogadas)

conn.close()

Columnas de ListaAhogamientos: ['IdAhogado', 'Fecha', 'Localidad', 'Provincia', 'Titular', 'Noticia', 'Enlace1', 'Enlace2', 'Enlace3', 'Enlace4', 'Enlace5', 'CCAA', 'Hora', 'Vigilancia', 'Latitud', 'Longitud', 'Maps', 'Actividad', 'Deteccion', 'Intervención', 'Localizacion', 'Riesgo', 'Campo1', 'Campo2', 'Tweet']
Columnas de PersonasAhogadas: ['IdPersona', 'Idahogado', 'Sexo', 'Edad', 'Nacionalidad', 'CCAA', 'Origen', 'Causa', 'FactorRiesgo', 'Antecedente', 'PrimerInterv', 'MatRescate', 'Extraccion', 'Tipo', 'Reanimacion', 'Pronostico', 'Notas']


Vemos que csv exportados estan en la bbdd

In [44]:
conn = sqlite3.connect("ahogamientos.db")

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]

print("Tablas en la base de datos:", tables)

conn.close()

Tablas en la base de datos: ['Aux-Actividad', 'Aux-Antecedentes', 'Aux-Causas', 'Aux-CCAA', 'Aux-Deteccion', 'Aux-Extraccion', 'Aux-FactorRiesgo', 'Aux-Intervencion', 'Aux-Localizacion', 'Aux-MaterialRescate', 'Aux-Origen', 'Aux-PrimerInterviniente', 'Aux-Pronostico', 'Aux-Provincia', 'Aux-Reanimacion', 'Aux-Riesgo', 'Aux-Tipo', 'AuxVigilancia', 'ListaAhogamientos', 'PersonasAhogadas']


OPCIONAL

query para formar la tabla lista Ahogamientos con sus relaciones

In [45]:
conn = sqlite3.connect("ahogamientos.db")

query = "SELECT * FROM [Aux-Provincia];"
df_aux_provincia = pd.read_sql_query(query, conn)

print(df_aux_provincia.head())

conn.close()

   IdProv Provincia  IdCCAA  PobProv  Costa Notas
0       1     Alava      13   326574      0  None
1       2  Albacete       7   390032      0  None
2       3  Alicante      10  1825332      1  None
3       4   Almería       1   706672      1  None
4       5     Avila       8   160700      0  None


In [46]:
conn = sqlite3.connect("ahogamientos.db")

query = "SELECT * FROM [ListaAhogamientos];"
df_aux_provincia = pd.read_sql_query(query, conn)

print(df_aux_provincia.head())

conn.close()

   IdAhogado             Fecha                  Localidad     Provincia  \
0        144  1/1/2013 0:00:00                    Iznajar       Córdoba   
1        146  9/1/2013 0:00:00          Puerto de la Cruz      Tenerife   
2        147  5/1/2013 0:00:00                   Arrecife     Lanzarote   
3        148  5/1/2013 0:00:00  San Bartolome de Tirajana  Gran Canaria   
4        150  6/1/2013 0:00:00           Playa del Inglés  Gran Canaria   

                                             Titular  \
0  Fallece un joven de 19 años al caer con su coc...   
1  Un alemán que llegó ayer a la Isla muere ahoga...   
2  Muere ahogado un alemán en una piscina del Pue...   
3  Un hombre de nacionalidad francesa fallece aho...   
4  Fallece una francés por un posible ahogamiento...   

                                             Noticia  \
0  Un joven de 19 años de edad, del que se descon...   
1   Un turista alemán, de 59 años, identificado c...   
2   Un hombre de 44 años de nacionalidad bri

In [47]:
conn = sqlite3.connect("ahogamientos.db")

query = """
SELECT 
    a.IdAhogado,
    a.Fecha,
    a.Localidad,
    a.Provincia,
    a.Titular,
    a.Noticia,
    a.Enlace1,
    a.Enlace2,
    a.Enlace3,
    a.Enlace4,
    a.Enlace5,
    c.CCAA,
    a.Hora,
    v.Vigilancia,
    a.Latitud,
    a.Longitud,
    a.Maps,
    act.Actividad,
    d.Deteccion,
    i.Intervencion,
    l.Localizacion,
    r.Riesgo,
    a.Campo1,
    a.Campo2,
    a.Tweet
FROM ListaAhogamientos a
LEFT JOIN [Aux-Provincia] p ON a.Provincia = p.IdProv
LEFT JOIN [Aux-CCAA] c ON a.CCAA = c.IdCCAA
LEFT JOIN [AuxVigilancia] v ON a.Vigilancia = v.IdVigilancia
LEFT JOIN [Aux-Actividad] act ON a.Actividad = act.IdActividad
LEFT JOIN [Aux-Deteccion] d ON a.Deteccion = d.IdDeteccion
LEFT JOIN [Aux-Intervencion] i ON a.Intervención = i.IdInterv
LEFT JOIN [Aux-Localizacion] l ON a.Localizacion = l.IdLocal
LEFT JOIN [Aux-Riesgo] r ON a.Riesgo = r.IdRiesgo;

"""

df_resultado1 = pd.read_sql_query(query, conn)

print(df_resultado1.head())

conn.close()


   IdAhogado             Fecha                  Localidad     Provincia  \
0        144  1/1/2013 0:00:00                    Iznajar       Córdoba   
1        146  9/1/2013 0:00:00          Puerto de la Cruz      Tenerife   
2        147  5/1/2013 0:00:00                   Arrecife     Lanzarote   
3        148  5/1/2013 0:00:00  San Bartolome de Tirajana  Gran Canaria   
4        150  6/1/2013 0:00:00           Playa del Inglés  Gran Canaria   

                                             Titular  \
0  Fallece un joven de 19 años al caer con su coc...   
1  Un alemán que llegó ayer a la Isla muere ahoga...   
2  Muere ahogado un alemán en una piscina del Pue...   
3  Un hombre de nacionalidad francesa fallece aho...   
4  Fallece una francés por un posible ahogamiento...   

                                             Noticia  \
0  Un joven de 19 años de edad, del que se descon...   
1   Un turista alemán, de 59 años, identificado c...   
2   Un hombre de 44 años de nacionalidad bri

In [48]:
df_resultado1

,IdAhogado,Fecha,Localidad,Provincia,Titular,Noticia,Enlace1,Enlace2,Enlace3,Enlace4,...,Longitud,Maps,Actividad,Deteccion,Intervencion,Localizacion,Riesgo,Campo1,Campo2,Tweet
0,144,1/1/2013 0:00:00,Iznajar,Córdoba,Fallece un joven de 19 años al caer con su coc...,"Un joven de 19 años de edad, del que se descon...",#http://www.abcdesevilla.es/andalucia/20130101...,http://www.abc.es/cordoba/20130104/sevp-buzos-...,http://www.abcdesevilla.es/andalucia/20130103/...,#https://www.diariodesevilla.es/sevilla/encuen...,...,-4.311497,None,Circulación en vehiculo,"Acompañante, familiar, amigo",Recuperación de cadaver,"Embalses, Pantanos",Malas condiciones meterológicas o del agua (Ba...,NaN,None,None
1,146,9/1/2013 0:00:00,Puerto de la Cruz,Tenerife,Un alemán que llegó ayer a la Isla muere ahoga...,"Un turista alemán, de 59 años, identificado c...",#http://www.laopinion.es/sucesos/2013/01/10/al...,#http://eldia.es/2013-01-10/SUCESOS/4-Muere-ah...,None,None,...,-16.546084,#http://goo.gl/maps/ai2Op#,Buceo a pulmon,Socorrista en servicio,Rescate por Socorristas o personas adiestradas...,Piscinas de equipamientos hoteleros y similares,"Excelentes condiciones, entornos cerrados o cu...",NaN,None,None
2,147,5/1/2013 0:00:00,Arrecife,Lanzarote,Muere ahogado un alemán en una piscina del Pue...,Un hombre de 44 años de nacionalidad británic...,http://www.canarias7.es/articulo.cfm?id=287910...,None,None,None,...,-13.635242,"#https://www.google.es/maps?q=28.925463,-13.63...",Natación recreativa,Ciudadano,Asistencia por transeúntes no adiestrados,Playas con vigilancia,Buenas condiciones metereológicas o del agua (...,NaN,None,None
3,148,5/1/2013 0:00:00,San Bartolome de Tirajana,Gran Canaria,Un hombre de nacionalidad francesa fallece aho...,Un hombre de 67 años de nacionalidad francesa ...,#http://www.europapress.es/sociedad/sucesos-00...,None,None,None,...,-15.57382,"#https://www.google.es/maps?q=27.760267,-15.57...",Natación recreativa,Socorrista en servicio,Rescate por Socorristas o personas adiestradas...,Playas con vigilancia,Buenas condiciones metereológicas o del agua (...,NaN,None,None
4,150,6/1/2013 0:00:00,Playa del Inglés,Gran Canaria,Fallece una francés por un posible ahogamiento...,Un hombre de 67 años y de origen francés ha fa...,#http://www.canarias7.es/articulo.cfm?id=287926#,None,None,None,...,-15.563854,"#https://www.google.es/maps?q=27.759146,-15.56...",Natación recreativa,Sin datos,Rescate y atención in situ por personal de sal...,Playas con vigilancia,Buenas condiciones metereológicas o del agua (...,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8681,10120,1/4/2021 0:00:00,"Gijón, Elogio del Horizonte",Asturias,Encuentran en Gijón el cadáver de una mujer qu...,Encuentran en Gijón el cadáver de una mujer qu...,#https://www.lavozdeasturias.es/noticia/gijon/...,#https://www.lne.es/gijon/2021/04/02/rescatan-...,#https://www.elcomercio.es/gijon/hallan-cadave...,None,...,-5.661990,None,Sin datos,Ciudadano,Recuperación de cadaver,Franja costera o alta mar,Sin datos,NaN,None,None
8682,10121,29/3/2021 0:00:00,"Alcalá de Guadaira,Torrondo, Río Guadaira",Sevilla,Complicado rescate de un ciclista herido tras ...,Complicado rescate de un ciclista herido tras ...,#https://www.sevillaactualidad.com/alcala-de-g...,#https://www.lavozdealcala.com/noticias/47118-...,None,None,...,-5.857281,None,Circulación en vehiculo,"Acompañante, familiar, amigo",Rescate por personal adiestrado con equipamiento,"Rios, canales y similares",Sin datos,NaN,None,None
8683,10122,4/4/2025 0:00:00,"Yaiza, Playa Blanca",Lanzarote,Rescatan a una persona con signos de ahogamien...,Rescatan a una persona con signos de ahogamien...,#https://www3.gobiernodecanarias.org/noticias/...,#https://diariodeavisos.elespanol.com/2025/04/...,#https://www.lavozdelanzarote.com/actualidad/s...,None,...,-13.709216,None,Natación recreativa,Ciudadano,Asistencia por transeúntes no adiestrados,Playas con vigilancia,Sin datos,NaN,None,None


In [49]:
columnas_df_resultado1 = df_resultado1.columns.tolist()
print(columnas_df_resultado1)

['IdAhogado', 'Fecha', 'Localidad', 'Provincia', 'Titular', 'Noticia', 'Enlace1', 'Enlace2', 'Enlace3', 'Enlace4', 'Enlace5', 'CCAA', 'Hora', 'Vigilancia', 'Latitud', 'Longitud', 'Maps', 'Actividad', 'Deteccion', 'Intervencion', 'Localizacion', 'Riesgo', 'Campo1', 'Campo2', 'Tweet']


OPCIONAL

query para formar la tabla Personas Ahogadas con sus relaciones

In [50]:
conn = sqlite3.connect("ahogamientos.db")

query = """
SELECT 
    p.IdPersona,
    p.Idahogado,
    p.Sexo,
    p.Edad,
    p.Nacionalidad,
    c.CCAA,
    o.Origen,
    cs.Causa,
    f.Factor AS FactorRiesgo,
    an.Antecedentes,
    pi.PrimerInterviniente,
    mr.MatRescate,
    ex.Extraccion,
    t.TipoAhogamiento,
    re.Reanimacion,
    pr.Pronostico,
    p.Notas
FROM PersonasAhogadas p
LEFT JOIN [Aux-CCAA] c ON p.CCAA = c.IdCCAA
LEFT JOIN [Aux-Origen] o ON p.Origen = o.IdOrigen
LEFT JOIN [Aux-Causas] cs ON p.Causa = cs.IdCausa
LEFT JOIN [Aux-FactorRiesgo] f ON p.FactorRiesgo = f.IdFactor
LEFT JOIN [Aux-Antecedentes] an ON p.Antecedente = an.IdAntecedentes
LEFT JOIN [Aux-PrimerInterviniente] pi ON p.PrimerInterv = pi.IdPrimerInterv
LEFT JOIN [Aux-MaterialRescate] mr ON p.MatRescate = mr.IdMatRescate
LEFT JOIN [Aux-Extraccion] ex ON p.Extraccion = ex.IdExtraccion
LEFT JOIN [Aux-Tipo] t ON p.Tipo = t.IdTipo
LEFT JOIN [Aux-Reanimacion] re ON p.Reanimacion = re.IdReanima
LEFT JOIN [Aux-Pronostico] pr ON p.Pronostico = pr.IdPronostico;
"""

df_resultado2 = pd.read_sql_query(query, conn)

print(df_resultado2.head())

conn.close()


   IdPersona  Idahogado    Sexo  Edad Nacionalidad        CCAA      Origen  \
0        166        144  Hombre  19.0       España   Andalucía       Local   
1        168        146  Hombre  59.0     Alemania  Extranjero  Extranjero   
2        169        147  Hombre  44.0  Reino Unido  Extranjero  Extranjero   
3        170        148  Hombre  67.0      Francia  Extranjero  Extranjero   
4        172        150  Hombre  67.0      Francia  Extranjero  Extranjero   

                                        Causa              FactorRiesgo  \
0  Accidente, caída o zambullida accidentales  Peligrosidad del entorno   
1                                   Sin datos                 Sin datos   
2                                   Sin datos                 Sin datos   
3                                   Sin datos                 Sin datos   
4                                   Sin datos                 Sin datos   

  Antecedentes           PrimerInterviniente MatRescate  \
0    Sin datos  Acomp

In [51]:
df_resultado2

,IdPersona,Idahogado,Sexo,Edad,Nacionalidad,CCAA,Origen,Causa,FactorRiesgo,Antecedentes,PrimerInterviniente,MatRescate,Extraccion,TipoAhogamiento,Reanimacion,Pronostico,Notas
0,166,144,Hombre,19.0,España,Andalucía,Local,"Accidente, caída o zambullida accidentales",Peligrosidad del entorno,Sin datos,"Acompañante, familiar, amigo",Sin datos,Cuerpo de Seguridad o Emergencias,Ahogamiento secundario a accidente,Recuperacion de cadaver,Ahogamiento mortal,None
1,168,146,Hombre,59.0,Alemania,Extranjero,Extranjero,Sin datos,Sin datos,Sin datos,Socorrista,Sin datos,Socorrista,Posible Ahogamiento,RCP basica SOS y SVA por SEM,Ahogamiento mortal,None
2,169,147,Hombre,44.0,Reino Unido,Extranjero,Extranjero,Sin datos,Sin datos,Sin datos,Ciudadano,Sin datos,Ciudadano,Ahogado con sintomas de aspiración de agua,SVA por SEM,Ahogamiento mortal,None
3,170,148,Hombre,67.0,Francia,Extranjero,Extranjero,Sin datos,Sin datos,Sin datos,Socorrista,Sin datos,Socorrista,Ahogado con sintomas de aspiración de agua,RCP basica SOS y SVA por SEM,Ahogamiento mortal,None
4,172,150,Hombre,67.0,Francia,Extranjero,Extranjero,Sin datos,Sin datos,Sin datos,Ciudadano,Sin datos,Socorrista,Posible Ahogamiento,RCP basica SOS y SVA por SEM,Ahogamiento mortal,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11501,12113,10120,Mujer,50.0,España,Asturias,Local,Suicidio,Sin datos,Sin datos,Ciudadano,Sin datos,Cuerpo de Seguridad o Emergencias,Posible Ahogamiento,Recuperacion de cadaver,Ahogamiento mortal,None
11502,12114,10121,Hombre,NaN,España,Andalucía,Local,Accidente de Trafico,Sin datos,Sin datos,"Acompañante, familiar, amigo",Sin datos,Cuerpo de Seguridad o Emergencias,Ahogamiento secundario a accidente,Sin Reanimacion,Ahogamiento no mortal con hospitalizacion,None
11503,12115,10122,Hombre,86.0,España,Canarias,Sin datos,Sin datos,"Edad, Enfermedad, Enfermedad Mental",Sin datos,Ciudadano,Sin datos,Ciudadano,Ahogado con sintomas de aspiración de agua,RCP basica temprana por transeunte adiestrado,Ahogamiento no mortal con hospitalizacion,None
11504,12116,10124,Hombre,84.0,Alemania,Extranjero,Extranjero,Sin datos,"Edad, Enfermedad, Enfermedad Mental",Sin datos,Ciudadano,Sin datos,Ciudadano,Ahogado con sintomas de aspiración de agua,RCP temprana por transeuntes no adiestrados,Ahogamiento no mortal con hospitalizacion,None


In [52]:
columnas_df_resultado2 = df_resultado2.columns.tolist()
print(columnas_df_resultado2)

['IdPersona', 'Idahogado', 'Sexo', 'Edad', 'Nacionalidad', 'CCAA', 'Origen', 'Causa', 'FactorRiesgo', 'Antecedentes', 'PrimerInterviniente', 'MatRescate', 'Extraccion', 'TipoAhogamiento', 'Reanimacion', 'Pronostico', 'Notas']


In [53]:
columnas_total = df_resultado1.columns.tolist() + df_resultado2.columns.tolist()
print(columnas_total)

['IdAhogado', 'Fecha', 'Localidad', 'Provincia', 'Titular', 'Noticia', 'Enlace1', 'Enlace2', 'Enlace3', 'Enlace4', 'Enlace5', 'CCAA', 'Hora', 'Vigilancia', 'Latitud', 'Longitud', 'Maps', 'Actividad', 'Deteccion', 'Intervencion', 'Localizacion', 'Riesgo', 'Campo1', 'Campo2', 'Tweet', 'IdPersona', 'Idahogado', 'Sexo', 'Edad', 'Nacionalidad', 'CCAA', 'Origen', 'Causa', 'FactorRiesgo', 'Antecedentes', 'PrimerInterviniente', 'MatRescate', 'Extraccion', 'TipoAhogamiento', 'Reanimacion', 'Pronostico', 'Notas']


CONSULTA FINAL PARA HACER LA TABLA COMPLETA

En esta query lo que hacemos es seleccionar todas las columnas de las tablas principales que son ListaAhogamientos y PersonasAhogadas y hacer un join entre ellas ya que queremos la info de las dos, y que estan relacionadas con IdAhogado

Para que todos los campos tengan un nombre y no un numero se hacen joins con las tablas auxiliares

In [54]:

conn = sqlite3.connect("ahogamientos.db")

query = """
SELECT 
    -- Datos de la tabla ListaAhogamientos
    [la].[IdAhogado],
    [la].[Fecha],
    [la].[Localidad],
    [la].[Provincia],
    [la].[Titular],
    [la].[Noticia],
    [la].[Enlace1],
    [la].[Enlace2],
    [la].[Enlace3],
    [la].[Enlace4],
    [la].[Enlace5],
    [ccaa].[CCAA],
    [la].[Hora],
    [vig].[Vigilancia],
    [la].[Latitud],
    [la].[Longitud],
    [la].[Maps],
    [act].[Actividad],
    [det].[Deteccion],
    [interv].[Intervencion],
    [loc].[Localizacion],
    [ries].[Riesgo],
    [la].[Campo1],
    [la].[Campo2],
    [la].[Tweet],

    -- Datos de la tabla PersonasAhogadas
    [pa].[IdPersona],
    [pa].[Sexo],
    [pa].[Edad],
    [pa].[Nacionalidad],
    [pa].[CCAA] AS CCAAPersona,
    [ori].[Origen],
    [cs].[Causa],
    [fr].[Factor] AS FactorRiesgo,
    [ant].[Antecedentes],
    [pi].[PrimerInterviniente],
    [mr].[MatRescate] AS MaterialRescate,
    [ex].[Extraccion],
    [tipo].[TipoAhogamiento],
    [rea].[Reanimacion],
    [pr].[Pronostico],
    [pa].[Notas]

FROM [ListaAhogamientos] AS [la]
LEFT JOIN [PersonasAhogadas] AS [pa] ON [la].[IdAhogado] = [pa].[Idahogado]

-- Joins auxiliares para ListaAhogamientos
LEFT JOIN [Aux-CCAA] AS [ccaa] ON [la].[CCAA] = [ccaa].[IdCCAA]
LEFT JOIN [AuxVigilancia] AS [vig] ON [la].[Vigilancia] = [vig].[IdVigilancia]
LEFT JOIN [Aux-Actividad] AS [act] ON [la].[Actividad] = [act].[IdActividad]
LEFT JOIN [Aux-Deteccion] AS [det] ON [la].[Deteccion] = [det].[IdDeteccion]
LEFT JOIN [Aux-Intervencion] AS [interv] ON [la].[Intervención] = [interv].[IdInterv]
LEFT JOIN [Aux-Localizacion] AS [loc] ON [la].[Localizacion] = [loc].[IdLocal]
LEFT JOIN [Aux-Riesgo] AS [ries] ON [la].[Riesgo] = [ries].[IdRiesgo]

-- Joins auxiliares para PersonasAhogadas
LEFT JOIN [Aux-Origen] AS [ori] ON [pa].[Origen] = [ori].[IdOrigen]
LEFT JOIN [Aux-Causas] AS [cs] ON [pa].[Causa] = [cs].[IdCausa]
LEFT JOIN [Aux-FactorRiesgo] AS [fr] ON [pa].[FactorRiesgo] = [fr].[IdFactor]
LEFT JOIN [Aux-Antecedentes] AS [ant] ON [pa].[Antecedente] = [ant].[IdAntecedentes]
LEFT JOIN [Aux-PrimerInterviniente] AS [pi] ON [pa].[PrimerInterv] = [pi].[IdPrimerInterv]
LEFT JOIN [Aux-MaterialRescate] AS [mr] ON [pa].[MatRescate] = [mr].[IdMatRescate]
LEFT JOIN [Aux-Extraccion] AS [ex] ON [pa].[Extraccion] = [ex].[IdExtraccion]
LEFT JOIN [Aux-Tipo] AS [tipo] ON [pa].[Tipo] = [tipo].[IdTipo]
LEFT JOIN [Aux-Reanimacion] AS [rea] ON [pa].[Reanimacion] = [rea].[IdReanima]
LEFT JOIN [Aux-Pronostico] AS [pr] ON [pa].[Pronostico] = [pr].[IdPronostico];
"""


df_resultado3 = pd.read_sql_query(query, conn)

print(df_resultado3.head())

conn.close()


   IdAhogado             Fecha                  Localidad     Provincia  \
0        144  1/1/2013 0:00:00                    Iznajar       Córdoba   
1        146  9/1/2013 0:00:00          Puerto de la Cruz      Tenerife   
2        147  5/1/2013 0:00:00                   Arrecife     Lanzarote   
3        148  5/1/2013 0:00:00  San Bartolome de Tirajana  Gran Canaria   
4        150  6/1/2013 0:00:00           Playa del Inglés  Gran Canaria   

                                             Titular  \
0  Fallece un joven de 19 años al caer con su coc...   
1  Un alemán que llegó ayer a la Isla muere ahoga...   
2  Muere ahogado un alemán en una piscina del Pue...   
3  Un hombre de nacionalidad francesa fallece aho...   
4  Fallece una francés por un posible ahogamiento...   

                                             Noticia  \
0  Un joven de 19 años de edad, del que se descon...   
1   Un turista alemán, de 59 años, identificado c...   
2   Un hombre de 44 años de nacionalidad bri

Pasamos la query a un csv para analizarlo

In [55]:
df_resultado3.to_csv("ahogamientos_completo.csv", sep=';', index=False, encoding='utf-8')

In [56]:
df_resultado3

,IdAhogado,Fecha,Localidad,Provincia,Titular,Noticia,Enlace1,Enlace2,Enlace3,Enlace4,...,Causa,FactorRiesgo,Antecedentes,PrimerInterviniente,MaterialRescate,Extraccion,TipoAhogamiento,Reanimacion,Pronostico,Notas
0,144,1/1/2013 0:00:00,Iznajar,Córdoba,Fallece un joven de 19 años al caer con su coc...,"Un joven de 19 años de edad, del que se descon...",#http://www.abcdesevilla.es/andalucia/20130101...,http://www.abc.es/cordoba/20130104/sevp-buzos-...,http://www.abcdesevilla.es/andalucia/20130103/...,#https://www.diariodesevilla.es/sevilla/encuen...,...,"Accidente, caída o zambullida accidentales",Peligrosidad del entorno,Sin datos,"Acompañante, familiar, amigo",Sin datos,Cuerpo de Seguridad o Emergencias,Ahogamiento secundario a accidente,Recuperacion de cadaver,Ahogamiento mortal,None
1,146,9/1/2013 0:00:00,Puerto de la Cruz,Tenerife,Un alemán que llegó ayer a la Isla muere ahoga...,"Un turista alemán, de 59 años, identificado c...",#http://www.laopinion.es/sucesos/2013/01/10/al...,#http://eldia.es/2013-01-10/SUCESOS/4-Muere-ah...,None,None,...,Sin datos,Sin datos,Sin datos,Socorrista,Sin datos,Socorrista,Posible Ahogamiento,RCP basica SOS y SVA por SEM,Ahogamiento mortal,None
2,147,5/1/2013 0:00:00,Arrecife,Lanzarote,Muere ahogado un alemán en una piscina del Pue...,Un hombre de 44 años de nacionalidad británic...,http://www.canarias7.es/articulo.cfm?id=287910...,None,None,None,...,Sin datos,Sin datos,Sin datos,Ciudadano,Sin datos,Ciudadano,Ahogado con sintomas de aspiración de agua,SVA por SEM,Ahogamiento mortal,None
3,148,5/1/2013 0:00:00,San Bartolome de Tirajana,Gran Canaria,Un hombre de nacionalidad francesa fallece aho...,Un hombre de 67 años de nacionalidad francesa ...,#http://www.europapress.es/sociedad/sucesos-00...,None,None,None,...,Sin datos,Sin datos,Sin datos,Socorrista,Sin datos,Socorrista,Ahogado con sintomas de aspiración de agua,RCP basica SOS y SVA por SEM,Ahogamiento mortal,None
4,150,6/1/2013 0:00:00,Playa del Inglés,Gran Canaria,Fallece una francés por un posible ahogamiento...,Un hombre de 67 años y de origen francés ha fa...,#http://www.canarias7.es/articulo.cfm?id=287926#,None,None,None,...,Sin datos,Sin datos,Sin datos,Ciudadano,Sin datos,Socorrista,Posible Ahogamiento,RCP basica SOS y SVA por SEM,Ahogamiento mortal,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11499,10120,1/4/2021 0:00:00,"Gijón, Elogio del Horizonte",Asturias,Encuentran en Gijón el cadáver de una mujer qu...,Encuentran en Gijón el cadáver de una mujer qu...,#https://www.lavozdeasturias.es/noticia/gijon/...,#https://www.lne.es/gijon/2021/04/02/rescatan-...,#https://www.elcomercio.es/gijon/hallan-cadave...,None,...,Suicidio,Sin datos,Sin datos,Ciudadano,Sin datos,Cuerpo de Seguridad o Emergencias,Posible Ahogamiento,Recuperacion de cadaver,Ahogamiento mortal,None
11500,10121,29/3/2021 0:00:00,"Alcalá de Guadaira,Torrondo, Río Guadaira",Sevilla,Complicado rescate de un ciclista herido tras ...,Complicado rescate de un ciclista herido tras ...,#https://www.sevillaactualidad.com/alcala-de-g...,#https://www.lavozdealcala.com/noticias/47118-...,None,None,...,Accidente de Trafico,Sin datos,Sin datos,"Acompañante, familiar, amigo",Sin datos,Cuerpo de Seguridad o Emergencias,Ahogamiento secundario a accidente,Sin Reanimacion,Ahogamiento no mortal con hospitalizacion,None
11501,10122,4/4/2025 0:00:00,"Yaiza, Playa Blanca",Lanzarote,Rescatan a una persona con signos de ahogamien...,Rescatan a una persona con signos de ahogamien...,#https://www3.gobiernodecanarias.org/noticias/...,#https://diariodeavisos.elespanol.com/2025/04/...,#https://www.lavozdelanzarote.com/actualidad/s...,None,...,Sin datos,"Edad, Enfermedad, Enfermedad Mental",Sin datos,Ciudadano,Sin datos,Ciudadano,Ahogado con sintomas de aspiración de agua,RCP basica temprana por transeunte adiestrado,Ahogamiento no mortal con hospitalizacion,None
11502,10124,4/4/2025 0:00:00,"Mogán, Playa de Amadores",Gran Canaria,Un bañista es sacado d

In [57]:
columnas_df_resultado3 = df_resultado3.columns.tolist()
print(columnas_df_resultado3)
print(columnas_total)

['IdAhogado', 'Fecha', 'Localidad', 'Provincia', 'Titular', 'Noticia', 'Enlace1', 'Enlace2', 'Enlace3', 'Enlace4', 'Enlace5', 'CCAA', 'Hora', 'Vigilancia', 'Latitud', 'Longitud', 'Maps', 'Actividad', 'Deteccion', 'Intervencion', 'Localizacion', 'Riesgo', 'Campo1', 'Campo2', 'Tweet', 'IdPersona', 'Sexo', 'Edad', 'Nacionalidad', 'CCAAPersona', 'Origen', 'Causa', 'FactorRiesgo', 'Antecedentes', 'PrimerInterviniente', 'MaterialRescate', 'Extraccion', 'TipoAhogamiento', 'Reanimacion', 'Pronostico', 'Notas']
['IdAhogado', 'Fecha', 'Localidad', 'Provincia', 'Titular', 'Noticia', 'Enlace1', 'Enlace2', 'Enlace3', 'Enlace4', 'Enlace5', 'CCAA', 'Hora', 'Vigilancia', 'Latitud', 'Longitud', 'Maps', 'Actividad', 'Deteccion', 'Intervencion', 'Localizacion', 'Riesgo', 'Campo1', 'Campo2', 'Tweet', 'IdPersona', 'Idahogado', 'Sexo', 'Edad', 'Nacionalidad', 'CCAA', 'Origen', 'Causa', 'FactorRiesgo', 'Antecedentes', 'PrimerInterviniente', 'MatRescate', 'Extraccion', 'TipoAhogamiento', 'Reanimacion', 'Prono

In [58]:
print(f"Filas: {df_resultado3.shape[0]}, Columnas: {df_resultado3.shape[1]}")


Filas: 11504, Columnas: 41


In [59]:
conn = sqlite3.connect("ahogamientos.db")

# Contar filas
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM PersonasAhogadas;")
num_rows = cursor.fetchone()[0]

# Contar columnas
cursor.execute("PRAGMA table_info('PersonasAhogadas');")
num_columns = len(cursor.fetchall())

print(f"Filas: {num_rows}, Columnas: {num_columns}")

conn.close()

Filas: 11506, Columnas: 17


In [60]:
# Obtener los IDs de la columna 'IdPersona' de ambas tablas
ids_personas_ahogadas = set(df['IdPersona'])
ids_resultado3 = set(df_resultado3['IdPersona'])

# Encontrar los IDs que están en 'PersonasAhogadas' pero no en 'df_resultado3'
ids_no_coinciden = ids_personas_ahogadas - ids_resultado3

print("IDs que no coinciden:", ids_no_coinciden)

IDs que no coinciden: {10517, 10470}
